## Introduction

### LangChain Components-LLMChain and Prompt template

**Langchain** is a Python library that allows you to create powerful applications using language models and prompts. **LLMChain** and **ChatPromptTemplate** are two of the main components of Langchain that enable you to generate natural language responses from various inputs.

An **LLMChain** is an object that combines a prompt template and a language model. A **prompt template** is a string that defines the format and structure of the input and output for the language model. A language model is a neural network that can generate natural language text based on some input. An LLMChain takes a dictionary of key-value pairs as input, formats the prompt template using the values, passes the formatted string to the language model, and returns the language model output as a response. You can use different language models such as OpenAI, GPT-3, or Llama 2 with LLMChain.


### Objective:

To call the GPT model API from Lang chain application for getting social media copy by sending input parameters "bio" and "transcript" using ChatPromptTemplate and LLMChain.


In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import TransformChain, LLMChain, SequentialChain
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

## Get podcast transcript created in Lab1

In [ ]:
# Open the file
with open('transcript.txt', 'r') as f:
    transcript = f.read()

# Print the content
print(transcript)

## Get biography created in Lab3

In [ ]:
# Open the file
with open('bio.txt', 'r') as f:
    bio = f.read()

# Print the content
print(bio)

## Connect to Azure OpenAI service

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")

In [ ]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
api_key = os.getenv("AZURE_OPENAI_API_KEY"),
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

model =  os.getenv("AZURE_OPENAI_CHAT_MODEL")
print(model)

## Put transcript and bio into the prompt

In [ ]:
system_template="You are a helpful large language model that can create a LinkedIn promo blurb for episodes of the podcast Behind the Tech, when given transcripts of the podcasts.  The Behind the Tech podcast is hosted by Kevin Scott.\n"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

user_prompt=PromptTemplate(
    template="Create a short summary of this podcast episode that would be appropriate to post on LinkedIn to promote the podcast episode.  The post should be from the first-person perspective of Kevin Scott, who hosts the podcast.\n" +
            "Here is the transcript of the podcast episode: {transcript} \n" +
            "Here is the bio of the guest: {bio} \n",
    input_variables=["transcript", "bio"],
)
human_message_prompt = HumanMessagePromptTemplate(prompt=user_prompt)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Get formatted messages for the chat completion
blurb_messages = chat_prompt.format_prompt(transcript={transcript}, bio={bio}).to_messages()
blurb_messages

## Send request to GPT model to create a LinkedIn post

In [ ]:
# Make a call to Azure OpenAI Service to get a social media blurb, 
print("Calling GPT-4 model on Azure OpenAI Service to get a social media blurb...\n")
gpt = AzureChatOpenAI(
    #openai_api_base=azure_endpoint,
    openai_api_version=api_version,
    deployment_name=model,
    #openai_api_key=api_key,
    openai_api_type = "azure",
)
#print(gpt4)   #shows parameters

output = gpt(blurb_messages)
social_media_post = output.content

gpt4_chain = LLMChain(llm=gpt, prompt=chat_prompt, output_key="social_media_copy")

print("Social Media Copy:\n")
print(social_media_post)
print("\n")


## Save Social media post to file

In [ ]:
# Specify the file path
file_path = "social_media_post.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(social_media_post)